## TEST

In [1]:
from Levenshtein import distance
import tensorflow as tf
import os

2024-05-09 16:39:36.494334: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 16:39:36.501233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 16:39:37.469895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-09 16:39:39.493029: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 16:39:49.828991: W tensorflow/compiler/tf2

In [2]:
dataset_path = 'dataset/transfer_dataset/'
val_dataset_path = os.path.join(dataset_path, 'val')

In [3]:
from inference_smartapp import handwriting_model
handwriting_model.model.summary()

models/keras/Model9v3_Words_Transfer
Loading pre-trained model and weights...


2024-05-09 16:40:12.858411: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 16:40:16.918246: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 16:40:16.918293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 16:40:16.920266: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2b:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 16:40:16.920306: I external/local_xla/xla/stream_executor

Model and weights loaded successfully.
Model: "handwriting_recognizer"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 image (InputLayer)          [(None, 1024, 128, 1)]       0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 1024, 128, 48)        480       ['image[0][0]']               
                                                                                                  
 Conv2 (Conv2D)              (None, 1024, 128, 96)        41568     ['Conv1[0][0]']               
                                                                                                  
 pool1 (MaxPooling2D)        (None, 512, 64, 96)          0         ['Conv2[0][0]']               
                                      

In [ ]:
val_df_list = os.listdir(val_dataset_path)
val_df_jpg_list = [val_df_list[i] for i in range(len(val_df_list)) if val_df_list[i].endswith('.jpg')]
val_df_jpg_list = [os.path.join(val_dataset_path, val_df_jpg_list[i]) for i in range(len(val_df_jpg_list)) ]

In [ ]:
scores = []
pred_real = []
for image_path in val_df_jpg_list:
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, 1)
    prediction = handwriting_model.inference(image)
    with open(image_path.replace('.jpg', '.txt'), 'r') as f:
        label = f.read().replace("|", " ")
    scores.append(distance(prediction, label))
    pred_real.append((prediction, label))
    #print(f'Prediction: {prediction} Label: {label} Distance: {scores[-1]}')


In [ ]:
print(f'Mean distance: {round(sum(scores) / len(scores),2)}')

In [ ]:
print("(Prediction,Real)")
for pred_r in pred_real:
    print(pred_r)